__OPTIMIZING MODEL PARAMETERS__

Now that we have a model and data it’s time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration (called an epoch) the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters (as we saw in the previous section), and optimizes these parameters using gradient descent. For a more detailed walkthrough of this process, check out this video on [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

/home/akorra/.local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

__Hyperparameters__

Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates ([read more about](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html) hyperparameter tuning)



We define the following hyperparameters for training:
<ul>
    <li><b>Number of Epochs</b> - the number times to iterate over the dataset</li>
    <li><b>Batch Size</b> - the number of data samples propagated through the network before the parameters are updated</li>
    <li><b>Learning Rate</b> - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.</li>
</ul>

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

__Optimization Loop__

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:
<ul>
    <li><b>The Train Loop</b> - iterate over the training dataset and try to converge to optimal parameters.</li>
    <li><b>The Validation/Test Loop</b> - iterate over the test dataset to check if model performance is improving.</li>
</ul>

__Loss Function__

When presented with some training data, our untrained network is likely not to give the correct answer. __Loss function__ measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include <code>nn.MSELoss</code> (Mean Square Error) for regression tasks, and <code>nn.NLLLoss</code> (Negative Log Likelihood) for classification. <code>nn.CrossEntropyLoss</code> combines <code>nn.LogSoftmax</code> and <code>nn.NLLLoss</code>.

We pass our model’s output logits to <code>nn.CrossEntropyLoss</code>, which will normalize the logits and compute the prediction error.

In [8]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

__Optimizer__

Optimization is the process of adjusting model parameters to reduce model error in each training step. __Optimization algorithms__ define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many [different optimizers](https://pytorch.org/docs/stable/optim.html) available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
<ul>
    <li>Call <code>optimizer.zero_grad()</code> to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.</li>
    <li>Backpropagate the prediction loss with a call to <code>loss.backwards()</code>. PyTorch deposits the gradients of the loss w.r.t. each parameter.</li>
    <li>Once we have our gradients, we call <code>optimizer.step()</code> to adjust the parameters by the gradients collected in the backward pass.</li>
</ul>

__Full Implementation__

In [10]:
# We define train_loop that loops over our optimization code, 
# and test_loop that evaluates the model’s performance against our test data.

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [11]:
# We initialize the loss function and optimizer, and pass it to train_loop and test_loop. 
# Feel free to increase the number of epochs to track the model’s improving performance.

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295915  [    0/60000]
loss: 2.293642  [ 6400/60000]
loss: 2.280433  [12800/60000]
loss: 2.285360  [19200/60000]
loss: 2.282107  [25600/60000]
loss: 2.253278  [32000/60000]
loss: 2.269715  [38400/60000]
loss: 2.246048  [44800/60000]
loss: 2.227989  [51200/60000]
loss: 2.232788  [57600/60000]
Test Error: 
 Accuracy: 30.1%, Avg loss: 2.228287 

Epoch 2
-------------------------------
loss: 2.215982  [    0/60000]
loss: 2.237209  [ 6400/60000]
loss: 2.199034  [12800/60000]
loss: 2.227150  [19200/60000]
loss: 2.219539  [25600/60000]
loss: 2.151775  [32000/60000]
loss: 2.200314  [38400/60000]
loss: 2.143411  [44800/60000]
loss: 2.110806  [51200/60000]
loss: 2.133810  [57600/60000]
Test Error: 
 Accuracy: 33.8%, Avg loss: 2.121686 

Epoch 3
-------------------------------
loss: 2.094414  [    0/60000]
loss: 2.144430  [ 6400/60000]
loss: 2.065641  [12800/60000]
loss: 2.129237  [19200/60000]
loss: 2.116353  [25600/60000]
loss: 1.997418  [32000/600